# Documentación para el sistema de adquisición de imagen.

Este sistema se encarga de localizar el micrófono en una imagen. Ha de ser suficientemente preciso como para minimizar el error y suficientemente rapido como para poder ejecutarse a 30fps, tiempo real.

*Nota*: la respuesta en frecuencia de la esfera (y por lo tanto el error que introduce en la medición del campo acustico) no se incluye en este documento. Consultar documentación correspondiente para encontrar esta información.

## Dimensiones de la esfera

- Diametro: 8 cm
- Tamaño de los radios: 12 mm

## Error de medición: Posición

Para medir este error usaremos una bateria de fotos tomadas con la cámara usada para el proyecto. Estas fotos serán etiquetadas manualmente y serán pasadas a través del sistema. Con la diferencia entre la "ground_truth" (gt a partir de ahora) y los resultados que devuelve nuestro sistema, calcularemos el error medido como la diferencia en pixeles de la posicion real y la adquirida.

In [1]:
import os
import json
import cv2
import numpy as np

module_path = os.path.abspath(os.path.join('..', '..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from app.package.services.mask import get_mask, get_circles
from app.package.services import imbasic as imb


In [2]:
# Get list of images
dir_img = os.path.join('imgs')
img_list = os.listdir(dir_img)

imgs = list(filter(lambda x: x.endswith('.png'), img_list))
# print(imgs)


In [3]:
def process_frame(frame):
    mask = get_mask(frame)
    return get_circles(mask), mask

In [4]:
dists = []
show = False

fn = 0

for index, img_name in enumerate(imgs):
    ann_path = os.path.join('tfg-microphone-detection', img_name+'.json')
    img_path = os.path.join('imgs', img_name)

    with open(ann_path) as json_file:
        data = json.load(json_file)
        if len(data['objects']) == 0:
            continue
        gt = data['objects'][0]['points']['exterior'][0]

    frame = cv2.imread(img_path)
    circles, mask = process_frame(frame)
    if circles is None:
        # False negative
        fn += 1
        print(f'fn in {img_name}')
        imb.imshow(frame, win_name='img')
        imb.imshow(mask, win_name='mask')
        cv2.waitKey(0)
        continue

    pt = circles[0][0][:2]
    if show and index < 10:
        frame = cv2.circle(frame, (gt[0], gt[1]), 2, (0, 255, 0), 4)
        x, y = [int(p) for p in pt]
        print(x, y)
        frame = cv2.rectangle(frame, (x - 2, y - 2), (x + 2, y + 2), (0, 128, 255), -1)
        imb.imshow(frame, win_name=str(index))
    
    error = gt-pt
    dist = np.sqrt(error[0]**2 + error[1]**2)
    dists.append(dist)

    
if show: 
    cv2.waitKey(0)
cv2.destroyAllWindows()

fn in 110.png
fn in 140.png
fn in 170.png
fn in 190.png
fn in 200.png
fn in 210.png
fn in 220.png
fn in 230.png
fn in 290.png
fn in 300.png
fn in 380.png
fn in 390.png
fn in 40.png
fn in 420.png
fn in 430.png
fn in 480.png
fn in 520.png
fn in 540.png
fn in 80.png
fn in 830.png


In [21]:
# print(dists)
print(f'Total of images: {len(imgs)}')
print(f'False negatives: {fn}')
print('Mean =', round(np.mean(dists), 2))
print('STD  =', round(np.std(dists), 2))
print('Max  =', round(np.max(dists), 2))
print('Min  =', round(np.min(dists), 2))

Total of images: 75
False negatives: 20
Mean = 8.42
STD  = 4.41
Max  = 22.64
Min  = 0.71
